### 라이브러리 설치

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
from sentence_transformers import SentenceTransformer

import numpy as np
import pandas as pd
import csv
import os

### 환경변수 및 api키 설정

In [ ]:
# 환경변수 불러오기
load_dotenv()

In [ ]:
# OpenAI API 키 설정 (환경 변수에서 가져오기)
openai_api_key = os.getenv('OPENAI_API_KEY')

#### File 불러오기

In [ ]:
# 파일 경로 지정
df = pd.read_csv('정보통신산업진흥원_SW산업정보종합시스템 SW제품정보_20221031.csv',encoding='949')
df.head()

### 데이터 전처리

In [ ]:
# 불필요 컬럼 제거 및 결측치 처리
df = df.drop(['제품코드', '홈페이지', '운영환경'], axis=1)
df = df.fillna('')

In [ ]:
# 불필요 방해 단어 제거
df['제품소개'] = df['제품소개'].str.replace('o ', ' ')
df['주요기능'] = df['주요기능'].str.replace('o ', ' ')

In [ ]:
# # 데이터 중복 제거
# df = df.drop_duplicates()

In [ ]:
df.head()

#### CSV 파일 Page 별 Split

In [ ]:
# 행별로 리스트화
df_list = df.values.tolist()

# 결과 확인
for row in df_list:
    print(row)

### csv -> 다큐먼트 형식으로 변환

In [ ]:
# 각 행을 청크로 나누기 및 다큐먼트 형식으로 변환
def split_row_into_chunks(row, max_length=1000):
    row_text = ' '.join(row)
    words = row_text.split()
    return [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]

documents = []
for row in df_list:
    chunks = split_row_into_chunks(row)
    row_documents = [Document(page_content=chunk) for chunk in chunks]
    documents.extend(row_documents)

In [ ]:
documents

#### Local FAISS에 Chunk 저장

In [ ]:
# 벡터 db 생성
embedding_model = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)

# 로컬에 db 저장
vectorstore.save_local('./db/faiss')